Q2(3)

In [5]:
pip install numpy


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.9 MB 12.2 MB/s eta 0:00:01
   ---------------- ----------------------- 5.2/12.9 MB 13.3 MB/s eta 0:00:01
   ------------------------- -------------- 8.4/12.9 MB 13.0 MB/s eta 0:00:01
   ----------------------------------- ---- 11.5/12.9 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 13.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\BANSAL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from itertools import product

dcs = list(range(1, 13))
demand_base = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}
scenarios = list(product([0.7, 1.0, 1.3], repeat=len(dcs)))

capacity = {1: 120, 2: 90, 3: 100}  # Total capacity is sum of all plant capacities
total_capacity = sum(capacity.values())

exceed_count = 0

for scenario in scenarios:
    total_demand = sum(demand_base[j] * scenario[j - 1] for j in dcs)
    if total_demand > total_capacity:
        exceed_count += 1

total_scenarios = len(scenarios)
fraction_exceeding = exceed_count / total_scenarios

print(f"Total scenarios: {total_scenarios}")
print(f"Scenarios where demand exceeds capacity: {exceed_count}")
print(f"Fraction of scenarios where demand exceeds total capacity: {fraction_exceeding:.4f}")


Total scenarios: 531441
Scenarios where demand exceeds capacity: 210724
Fraction of scenarios where demand exceeds total capacity: 0.3965


Q2(4)

In [21]:
import numpy as np
from itertools import product
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

factories = [1, 2, 3]
dcs = list(range(1, 13))
n_scen = 10
all_scen = list(product(['low', 'medium', 'high'], repeat=12))
sample_indices = np.random.choice(range(len(all_scen)), size=n_scen, replace=False)

scen_data = []
for idx in sample_indices:
    scn = {}
    for j in dcs:
        r = np.random.rand()
        med = {1: 200000, 2: 300000, 3: 400000, 4: 350000, 5: 150000, 6: 190000, 
               7: 250000, 8: 200000, 9: 180000, 10: 120000, 11: 300000, 12: 400000}[j]
        if r < 0.3:
            scn[j] = int(0.7 * med)
        elif r < 0.7:
            scn[j] = med
        else:
            scn[j] = int(1.3 * med)
    scen_data.append(scn)

scen_prob = 1.0 / n_scen

cap = {1: 1200000, 2: 900000, 3: 1000000}
cost_prod = {1: 8, 2: 7, 3: 9}
selling_price = 20
salvage_val = 5
penalty_val = 7
ship_cost = {
    (1, 1): 2, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 5, (1, 6): 4.4, 
    (1, 7): 3.2, (1, 8): 3, (1, 9): 1.5, (1, 10): 3, (1, 11): 6.0, (1, 12): 1.0,
    (2, 1): 1.9, (2, 2): 3, (2, 3): 2.4, (2, 4): 2.4, (2, 5): 2, (2, 6): 4.1, 
    (2, 7): 2.8, (2, 8): 3, (2, 9): 1.7, (2, 10): 2, (2, 11): 1.0, (2, 12): 4.5,
    (3, 1): 3.1, (3, 2): 1.8, (3, 3): 2.8, (3, 4): 3.4, (3, 5): 2, (3, 6): 2.9, 
    (3, 7): 2.2, (3, 8): 3.2, (3, 9): 2, (3, 10): 2.8, (3, 11): 4.5, (3, 12): 6.0
}

model = LpProblem("Stochastic_Optimization", LpMaximize)

prod = {i: LpVariable(f'prod_{i}', lowBound=0, upBound=cap[i]) for i in factories}
ship = {(s, i, j): LpVariable(f'ship_{s}_{i}_{j}', lowBound=0) 
        for s in range(n_scen) for i in factories for j in dcs}
unsold = {(s, i): LpVariable(f'unsold_{s}_{i}', lowBound=0) 
          for s in range(n_scen) for i in factories}
lost = {(s, j): LpVariable(f'lost_{s}_{j}', lowBound=0) 
        for s in range(n_scen) for j in dcs}

model += lpSum(
    scen_prob * (
        lpSum(selling_price * ship[s, i, j] for i in factories for j in dcs) -
        lpSum(cost_prod[i] * prod[i] for i in factories) -
        lpSum(ship_cost[i, j] * ship[s, i, j] for i in factories for j in dcs) -
        lpSum(salvage_val * unsold[s, i] for i in factories) -
        lpSum(penalty_val * lost[s, j] for j in dcs)
    ) for s in range(n_scen)
)

for i in factories:
    model += prod[i] <= cap[i]

for s in range(n_scen):
    for i in factories:
        model += lpSum(ship[s, i, j] for j in dcs) + unsold[s, i] <= prod[i]

for s in range(n_scen):
    for j in dcs:
        model += lpSum(ship[s, i, j] for i in factories) + lost[s, j] == scen_data[s][j]

model.solve()

if model.status == 1:
    print(f"\nOptimal Expected Profit: {value(model.objective)}")
    print("\nOptimal Production Quantities:")
    for i in factories:
        print(f"Factory {i}: {prod[i].varValue} units")
    
    print("\nSample Shipment Plan for Scenario 0:")
    for i in factories:
        for j in dcs:
            if ship[0, i, j].varValue > 0:
                print(f"Ship {ship[0, i, j].varValue} units from Factory {i} to DC {j}")

    print("\nSample Unsold for Scenario 0:")
    for i in factories:
        if unsold[0, i].varValue > 0:
            print(f"Factory {i} unsold: {unsold[0, i].varValue} units")

    print("\nSample Lost Sales for Scenario 0:")
    for j in dcs:
        if lost[0, j].varValue > 0:
            print(f"Destination {j} lost sales: {lost[0, j].varValue} units")
else:
    print("No optimal solution found.")



Optimal Expected Profit: 27783164.61

Optimal Production Quantities:
Factory 1: 1200000.0 units
Factory 2: 900000.0 units
Factory 3: 963998.0 units

Sample Shipment Plan for Scenario 0:
Ship 145000.0 units from Factory 1 to DC 1
Ship 280000.0 units from Factory 1 to DC 3
Ship 140000.0 units from Factory 1 to DC 8
Ship 180000.0 units from Factory 1 to DC 9
Ship 400000.0 units from Factory 1 to DC 12
Ship 115000.0 units from Factory 2 to DC 1
Ship 455000.0 units from Factory 2 to DC 4
Ship 120000.0 units from Factory 2 to DC 10
Ship 210000.0 units from Factory 2 to DC 11
Ship 210000.0 units from Factory 3 to DC 2
Ship 105000.0 units from Factory 3 to DC 5
Ship 133000.0 units from Factory 3 to DC 6
Ship 325000.0 units from Factory 3 to DC 7

Sample Unsold for Scenario 0:

Sample Lost Sales for Scenario 0:


Q2(5)

Production Levels:The deterministic model produces a fixed amount based on medium demand levels.
The stochastic model adjusts production considering uncertainty, possibly reducing overproduction losses.

Profit:The deterministic model achieves a higher theoretical profit.
The stochastic model incorporates risk management, leading to a potentially more robust strategy.

Flexibility:The stochastic model provides flexibility by accounting for multiple demand scenarios.
The deterministic model assumes fixed demand, which may lead to higher losses if demand deviates.

The values of the two models are as follows:
Ques 1: Deterministic LP - 
Max Profit: 3,065,200.00 Rupees
Plant 1: 1200000 units
Plant 2: 900000 units
Plant 3: 940000 units

Ques 2 : Stochastic LP
Expected Profit: 27783164.6
Optimal Production Quantities:
Plant 1: 1200000.0 units
Plant 2: 900000.0 units
Plant 3: 963998.0 units